# 2D Acoustic Wave Equation
---

Consider the 2d acoustic wave equation
$$
\begin{aligned}
&\frac{\partial p}{\partial t} = -c^2(x, z)\rho(x, z)\left(\frac{\partial u}{\partial x} + \frac{\partial v}{\partial z}\right) + f(x, z, t) \\
&\frac{\partial u}{\partial t} = -\frac{1}{\rho(x, z)}\frac{\partial p}{\partial x} \\
&\frac{\partial v}{\partial t} = -\frac{1}{\rho(x, z)}\frac{\partial p}{\partial z},
\end{aligned}
$$
where $p$ is Pressure, $c$ is Medium Velocity. The source term $f$ is represented by a Ricker wavelet (practically, we regard it as a multiplication of Ricker wavelet in temporal domain with Gaussian dissolve in the spatial domain)
$$
f(x, z, t) = R(t)N(x, z),
$$
where
$$
R(t) = \left(1-2(\pi f_0(t-t_0))^2\right)\exp\left(-(\pi f_0(t-t_0))^2\right),
$$
where $f_0 = 20$ is the dominant frequency, $t_0 = 0.05$ is time delay, and
$$
N(x, z) = \exp\left(-\frac{1}{\alpha^2}\left((x-x_s)^2+(z-z_s)^2\right)\right).
$$


---

## Data

- $p$: Pressure

- All known data are collected in the spatial domain $(x, z) \in [1, 500]\times [1, 500]$, with grid size 10 (which means the spatial grids are {10, 20, ..., 500}. The temporal domain is $t\in [0, 0.5)$, with time step $dt = 0.01$. This gives 50 time steps as recorded, i.e., $t = \{0, 0.01, \cdots, 0.49\}$.

- The sources are located at $(x_s, z_s) = \{50, 100, \cdots, 450\} \times \{10\}$.

## Target
- Recover the direct problem $p$
- Recover the inverse problem $c$

## Scaling (A simple substitution of variables)

We let
$$
x' = \frac{x}{x_0}, \ z' = \frac{z}{z_0}, \ t' = \frac{t}{t_0}, \ u' = \frac{u}{u_0}, \ c' = \frac{c}{c_0},
$$
where $x_0, z_0, t_0, u_0, c_0$ are scaling constants to be determined. Then the equation becomes
$$
\frac{p_0}{t_0^2}\frac{\partial^2 p'}{\partial t'^2} = c_0^2c'(x_0x', z_0z')\left(\frac{p_0}{x_0^2}\frac{\partial^2 p'}{\partial x'^2} + \frac{p_0}{z_0^2}\frac{\partial^2 p'}{\partial z'^2}\right) + f(x_0x', z_0z', t_0t').
$$

In this case, we let
$$
x_0 = z_0 = c_0 = 1000, \ t_0 = 1, \ p_0 = 1,
$$

---

## Models

The inverse model will return just either values of $c$, which needs to be learned.

In [ ]:
NAME = "0707_two_values_of_c_first_order_equation"

In [10]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt
import time

import sys
sys.path.append("../../")
from Seismic_wave_inversion_PINN.tf_model_utils import *
from Seismic_wave_inversion_PINN.data_utils import *

plt.rcParams.update(plt.rcParamsDefault)
tf.keras.backend.set_floatx("float32")

%load_ext autoreload
%autoreload 2

print(tf.config.list_physical_devices('GPU'))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [11]:
def siren_model(layers, c, w0, lambda_1, adaptive = None):
	class scaled_dense(keras.layers.Layer):
		def __init__(self, units, input_dim, c, w0, lambda_1):
			super(scaled_dense, self).__init__()
			self.lambda_1 = lambda_1
			w_init = tf.random_uniform_initializer(-np.sqrt(c/input_dim), np.sqrt(c/input_dim))
			self.w = tf.Variable(initial_value = w_init(shape = (input_dim, units), dtype = "float32"), trainable = True)
			b_init = tf.zeros_initializer()
			self.b = tf.Variable(initial_value = b_init(shape=(units,), dtype="float32"), trainable=True)
			self.w0 = tf.Variable(w0, dtype = "float32", trainable = True)
			
		def call(self, inputs):
			self.add_loss(self.lambda_1*tf.reduce_sum(tf.abs(self.w)))
			return tf.sin(tf.matmul(inputs, self.w)*self.w0 + self.b)
	
	with tf.device("/device:GPU:0"):
		model = keras.models.Sequential()
		if adaptive == "first" or adaptive == "all":
			model.add(scaled_dense(layers[1], layers[0], c, w0, lambda_1))
		else:
			model.add(keras.layers.Dense(layers[1], input_shape = (layers[0], ), activation = K.sin,
										kernel_initializer = keras.initializers.RandomUniform(-w0*np.sqrt(c/layers[0]), w0*np.sqrt(c/layers[0])),
										kernel_regularizer = keras.regularizers.l1(lambda_1)))
		for i in range(1, len(layers)-2):
			if adaptive == "all":
				model.add(scaled_dense(layers[i+1], layers[i], c, 1.0, lambda_1))
			else:
				model.add(keras.layers.Dense(layers[i+1], input_shape = (layers[i], ), activation = K.sin,
									kernel_initializer = keras.initializers.RandomUniform(-np.sqrt(c/layers[i]), np.sqrt(c/layers[i])),
									kernel_regularizer = keras.regularizers.l1(lambda_1)))
		model.add(keras.layers.Dense(layers[-1]))
	return model

def tanh_model(layers, adaptive = None, bias = None):
	class adaptive_dense(keras.layers.Layer):
		def __init__(self, units, input_dim):
			super(adaptive_dense, self).__init__()
			w_init = keras.initializers.GlorotUniform()
			self.w = tf.Variable(initial_value = w_init(shape = (input_dim, units), dtype = "float32"), trainable = True)
			b_init = tf.zeros_initializer()
			self.b = tf.Variable(initial_value = b_init(shape=(units,), dtype="float32"), trainable=True)
			self.a = tf.Variable(1.0, dtype = "float32", trainable = True)
			
		def call(self, inputs):
			return tf.tanh(self.a*(tf.matmul(inputs, self.w) + self.b))
	
	with tf.device("/device:GPU:0"):
		model = keras.models.Sequential()
		if adaptive == "first" or adaptive == "all":
			model.add(adaptive_dense(layers[1], layers[0]))
		else:
			model.add(keras.layers.Dense(layers[1], input_shape = (layers[0], ), activation = "tanh"))
		for i in range(1, len(layers)-2):
			if adaptive == "all":
				model.add(adaptive_dense(layers[i+1], layers[i]))
			else:
				model.add(keras.layers.Dense(layers[i+1], input_shape = (layers[i], ), activation = "tanh"))
		if bias:
			model.add(keras.layers.Dense(layers[-1], bias_initializer = keras.initializers.Constant(bias)))
		else:
			model.add(keras.layers.Dense(layers[-1]))
	return model

class rectangular_constant_model(keras.Model):
	def __init__(self, boundary, value):
		"""
		boundary: the interfaces of the middle rectangular interval; in ascending order; len(boundary) = 2
		value_init: initial values of each variable; len(value_init) = 2; here we suppose the value in the first and third interval are the same
		"""
		super(rectangular_constant_model, self).__init__()
		self.boundary = boundary
		self.values = [tf.Variable(vi, dtype = tf.float32, trainable = True) for vi in value]
		
	def call(self, inputs): 
		"""
		inputs: (x, z)
		outputs: c'(z)
		"""
		return tf.where(tf.math.logical_and(tf.greater_equal(inputs[:, 1:2], self.boundary[0]), 
											tf.less_equal(inputs[:, 1:2], self.boundary[1])),
						self.values[1],
						self.values[0])

w0 = 100
c = 6
lambda_1 = 0.5

direct_layers = [4, 64, 64, 64, 64, 3] # (x, z, t, xs) -> p
direct_model = siren_model(direct_layers, c, w0, lambda_1)

x0, z0, t0 = 1e3, 1e3, 1.0
p0, u0, v0 = 1.0, 1.0, 1.0
c0 = 1e3

boundary = np.array([200.0, 300.0]) / z0
inverse_values = np.array([1000, 1000]) / c0
inverse_model = rectangular_constant_model(boundary, inverse_values)

In [12]:
x0, z0, t0 = 1e3, 1e3, 1.0
p0, u0, v0 = 1.0, 1.0, 1.0
c0 = 1e3

# source
f_0 = 20.0*np.pi
alpha = 2.0*10 # multiplied by dx
M0 = 1 # scale p

@tf.function
def R(t, t_s):
	return M0*(1-2*(f_0*(t-t_s))**2)*tf.exp(-(f_0*(t-t_s))**2)

@tf.function
def N(x, z, x_s, z_s):
	return tf.exp(-1.0/(alpha**2) * ((x-x_s)**2 + (z-z_s)**2))

@tf.function
def f(x, z, t, x_s, z_s, t_s):
	return R(t, t_s)*N(x, z, x_s, z_s)

In [13]:
w_zero = 1e-2
threshold = 1e-3

def weighted_loss(true, pred):
	error = K.square(true - pred)
	error = K.mean(K.switch(K.less_equal(K.abs(true), threshold), w_zero * error , error))
	return error 

loss_func = keras.losses.MeanSquaredError()
loss_func_r = keras.losses.MeanAbsoluteError() # l1 loss

def get_residue_loss(tape, p, u, v, c, x, z, t, x_s, z_s, t_s):
	dp_dt = tape.gradient(p, t)
	dp_dx = tape.gradient(p, x)
	dp_dz = tape.gradient(p, z)
	du_dt = tape.gradient(u, t)
	du_dx = tape.gradient(u, x)
	dv_dt = tape.gradient(v, t)
	dv_dz = tape.gradient(v, z)
	eq1 = loss_func_r(p0/t0*dp_dt, -c0**2*tf.square(c)*(u0/x0*du_dx+v0/z0*dv_dz)+f(x0*x, z0*z, t0*t, x0*x_s, z0*z_s, t0*t_s))
	eq2 = loss_func_r(u0/t0*du_dt, -1.0*p0/x0*dp_dx)
	eq3 = loss_func_r(v0/t0*dv_dt, -1.0*p0/z0*dp_dz)
	return eq1, eq2, eq3

def step(data, residue = True):
	with tf.GradientTape(persistent = True) as tape:
		tape.watch(data)
		[x_c, z_c, t_c, x_sc, z_sc, t_sc,
		 x_i, z_i, t_i, p_i, u_i, v_i, x_si, z_si, t_si,
		 x_d, z_d, t_d, p_d, x_sd, z_sd, t_sd] = data
		if residue:
			out_c = direct_model(tf.concat([x_c, z_c, t_c, x_sc], axis = 1))
			c_c = inverse_model(tf.concat([x_c, z_c], axis = 1))
			out_d = direct_model(tf.concat([x_d, z_d, t_d, x_sd], axis = 1))
			c_d = inverse_model(tf.concat([x_d, z_d], axis = 1))
			out_i = direct_model(tf.concat([x_i, z_i, t_i, x_si], axis = 1))
			c_i = inverse_model(tf.concat([x_i, z_i], axis = 1))
			loss_c1, loss_c2, loss_c3 = get_residue_loss(tape, out_c[:, 0:1], out_c[:, 1:2], out_c[:, 2:3], c_c, x_c, z_c, t_c, x_sc, z_sc, t_sc)
			loss_dr1, loss_dr2, loss_dr3 = get_residue_loss(tape, out_d[:, 0:1], out_d[:, 1:2], out_d[:, 2:3], c_d, x_d, z_d, t_d, x_sd, z_sd, t_sd)
			loss_dv = weighted_loss(out_d[:, 0:1], p_d)
			loss_ir1, loss_ir2, loss_ir3 = get_residue_loss(tape, out_i[:, 0:1], out_i[:, 1:2], out_i[:, 2:3], c_i, x_i, z_i, t_i, x_si, z_si, t_si)
			loss_iv = loss_func(out_i[:, 0:1], p_i) + loss_func(out_i[:, 1:2], u_i) + loss_func(out_i[:, 2:3], v_i)
		else:
			out_d = direct_model(tf.concat([x_d, z_d, t_d, x_sd], axis = 1))
			out_i = direct_model(tf.concat([x_i, z_i, t_i, x_si], axis = 1))
			loss_c1, loss_c2, loss_c3 = 0, 0, 0
			loss_dr1, loss_dr2, loss_dr3 = 0, 0, 0
			loss_ir1, loss_ir2, loss_ir3 = 0, 0, 0
			loss_dv = weighted_loss(out_d[:, 0:1], p_d)
			loss_iv = loss_func(out_i[:, 0:1], p_i) + loss_func(out_i[:, 1:2], u_i) + loss_func(out_i[:, 2:3], v_i)

		loss = w_c1*loss_c1 + w_c2*loss_c2 + w_c3*loss_c3 + w_dr1*loss_dr1 + w_dr2*loss_dr2 + w_dr3*loss_dr3 + \
				w_dv*loss_dv + w_ir1*loss_ir1 + w_ir2*loss_ir2 + w_ir3*loss_ir3 + w_iv*loss_iv
	
	if residue:
		grads = tape.gradient(loss, direct_model.trainable_variables + inverse_model.trainable_variables)
		optimizer.apply_gradients(zip(grads, direct_model.trainable_variables + inverse_model.trainable_variables))
	else:
		grads = tape.gradient(loss, direct_model.trainable_variables)
		optimizer.apply_gradients(zip(grads, direct_model.trainable_variables))
	
	del tape
	return loss, loss_c1, loss_c2, loss_c3, loss_dr1, loss_dr2, loss_dr3, loss_dv, loss_ir1, loss_ir2, loss_ir3, loss_iv


# data_c: x_c, z_c, t_c, xs_c, z_sc, t_sc
# data_i: x_i, z_i, t_i, p_i, u_i, v_i, x_si, z_si, t_si
# data_d: x_d, z_d, t_d, p_d, x_sd, z_sd, t_sd
def train(residue, data_c, data_i, data_d, start_epoch, epochs, batch_proportion = 0.1, print_every = 100, save_every = 10000, save_path = None):
	shuffled = lambda dataset: [tf.data.Dataset.from_tensor_slices(tuple(d)).shuffle(buffer_size = max(d[0].shape[0], 1), \
                                                        reshuffle_each_iteration = True).batch(max(int(batch_proportion*d[0].shape[0]), 1)) \
                                        for d in dataset]
	
	data_s = shuffled([data_c, data_i, data_d])
	for epoch in range(start_epoch+1, start_epoch+epochs+1):
		losses = np.zeros((12,))
		for dc, di, dd in zip(*data_s):
			loss = step(list(dc) + list(di) + list(dd), residue)
			losses += np.array(loss)
		losses *= batch_proportion
		
		if epoch % print_every == 0:
			print("{}, Epoch: {}, Loss: {:.4e}, c1: {:.4e}, c2: {:.4e}, c3: {:.4e}, dr1: {:.4e}, dr2: {:.4e}, dr3: {:.4e}, \
					  dv: {:.4e}, ir1: {:.4e}, ir2: {:.4e}, ir3: {:.4e}, iv: {:.4e}".format(get_time(), epoch, *list(losses)))
		if epoch % save_every == 0:
			direct_model.save("models/{}/{}/checkpoint_{}/direct_model".format(NAME, save_path, epoch))
			inverse_model.save("models/{}/{}/checkpoint_{}/inverse_model".format(NAME, save_path, epoch))

In [14]:
from scipy.io import loadmat

x_s = np.linspace(50.0, 450.0, 9) / x0
z_s = np.array([10.0]) / z0
t_s = np.array([0.05]) / t0
n_source = len(x_s)

domain = [0, 500]
T_max = 0.5
dx = 10
dt = 0.01

In [15]:
# n_cx = n_cz = n_ct = 50 # 50*50*50
# x_c = np.linspace(*domain, n_cx) / x0
# z_c = np.linspace(*domain, n_cz) / z0
# t_c = np.linspace(0, T_max, n_ct) / t0
# xzts_c = tensor_grid([x_c, z_c, t_c, x_s, z_s, t_s])
# x_c, z_c, t_c, x_sc, z_sc, t_sc = xzts_c[:, 0:1], xzts_c[:, 1:2], xzts_c[:, 2:3], xzts_c[:, 3:4], xzts_c[:, 4:5], xzts_c[:, 5:6]

# n_cs = 1000
# r = transform(np.random.random((n_cs, 1)), 0, dx) / x0
# theta = transform(np.random.random((n_cs, 1)), 0, 2*np.pi)
# x_cs = np.vstack([xi + r*np.cos(theta) for xi in x_s])
# z_cs = np.vstack([z_s[0] + r*np.sin(theta) for i in x_s])
# t_cs = np.vstack([transform(np.random.random((n_s, 1)), t_s[0] - dt/t0, t_s[0] + dt/t0) for i in x_s])
# x_scs = np.tile(x_s.reshape((-1, 1)), (1, n_cs)).reshape((-1, 1))
# z_scs = np.tile(z_s, (n_source, n_cs)).reshape((-1, 1))
# t_scs = np.tile(t_s, (n_source, n_cs)).reshape((-1, 1))
# x_c = np.vstack([x_c, x_cs])
# z_c = np.vstack([z_c, z_cs])
# t_c = np.vstack([t_c, t_cs])
# x_sc = np.vstack([x_sc, x_scs])
# z_sc = np.vstack([z_sc, z_scs])
# t_sc = np.vstack([t_sc, t_scs])

# n_i = 2000
# x_i = np.vstack([transform(np.random.random((n_i, 1)), *domain) / x0 for xi in x_s])
# z_i = np.vstack([transform(np.random.random((n_i, 1)), *domain) / z0 for i in x_s])
# t_i = np.zeros_like(x_i) / t0
# p_i = np.zeros_like(x_i) / p0
# u_i = np.zeros_like(x_i) / u0
# v_i = np.zeros_like(x_i) / v0
# x_si = np.tile(x_s.reshape((-1, 1)), (1, n_i)).reshape((-1, 1))
# z_si = np.tile(z_s, (n_source, n_i)).reshape((-1, 1))
# t_si = np.tile(t_s, (n_source, n_i)).reshape((-1, 1))

# map_to_tf_float32 = lambda x: list(map(lambda y: tf.constant(y, dtype = tf.float32), x))

# n_d = 2500
# p_d = []
# for x in x_s:
# 	p = loadmat("data/sr_{}.mat".format(int(x*x0/dx)))["sr_{}".format(int(x*x0/dx))].reshape((-1, 1)) / p0
# 	p_d.append(p)
	
# p_d = np.vstack(p_d)
# x_d = np.linspace(10.0, 500.0, 50) / x0
# z_d = np.array([10.0]) / z0
# t_d = np.linspace(0.0, 0.49, 50) / t0
# txzs_d = tensor_grid([t_d, x_d, z_d, x_s, z_s, t_s])
# x_d, z_d, t_d, x_sd, z_sd, t_sd = txzs_d[:, 1:2], txzs_d[:, 2:3], txzs_d[:, 0:1], txzs_d[:, 3:4], txzs_d[:, 4:5], txzs_d[:, 5:6]

# data_c = map_to_tf_float32([x_c, z_c, t_c, x_sc, z_sc, t_sc])
# data_i = map_to_tf_float32([x_i, z_i, t_i, p_i, u_i, v_i, x_si, z_si, t_si])
# data_d = map_to_tf_float32([x_d, z_d, t_d, p_d, x_sd, z_sd, t_sd])

# from six.moves import cPickle as pickle

# with open("dataset_{}.pkl".format(NAME), "wb") as f:
# 	pickle.dump([data_c, data_i, data_d], f)

In [16]:
from six.moves import cPickle as pickle

with open("dataset_{}.pkl".format(NAME), "rb") as file:
	data_c, data_i, data_d = pickle.load(file)

In [ ]:
i = 1
optimizer = keras.optimizers.Adam(learning_rate = keras.optimizers.schedules.PiecewiseConstantDecay([1000], 
										[1e-2, 1e-3]))

w_c1, w_c2, w_c3 = 1.0, 1.0, 1.0
w_dr1, w_dr2, w_dr3 = 1.0, 1.0, 1.0
w_ir1, w_ir2, w_ir3 = 1.0, 1.0, 1.0
w_dv = 1e8
w_iv = 1e2

w_zero = 0.01
threshold = 1e-4

train(True, data_c, data_i, data_d, 
      start_epoch = 0, epochs = 3000, 
      batch_proportion = 0.1, print_every = 2, save_every = 50, save_path = "{}_adam".format(i))

2020/07/11, 19:18:28, Epoch: 2, Loss: 3.1461e+06, c1: 2.7221e+04, c2: 2.9132e+01, c3: 2.7789e+01, dr1: 2.7077e+04, dr2: 2.8952e+01, dr3: 2.7435e+01, 					  dv: 3.0646e-02, ir1: 2.7037e+04, ir2: 2.9036e+01, ir3: 2.7976e+01, iv: 2.3339e-01
2020/07/11, 19:19:08, Epoch: 4, Loss: 4.7378e+05, c1: 4.0724e+03, c2: 6.2822e+00, c3: 5.3878e+00, dr1: 4.0465e+03, dr2: 6.2586e+00, dr3: 5.3854e+00, 					  dv: 4.6159e-03, ir1: 4.0329e+03, ir2: 6.2735e+00, ir3: 5.3599e+00, iv: 1.4719e-02
2020/07/11, 19:19:47, Epoch: 6, Loss: 1.7095e+05, c1: 9.2857e+02, c2: 1.4180e+00, c3: 1.6428e+00, dr1: 9.2018e+02, dr2: 1.4272e+00, dr3: 1.6313e+00, 					  dv: 1.6815e-03, ir1: 9.4157e+02, ir2: 1.4098e+00, ir3: 1.6308e+00, iv: 2.4114e-03
2020/07/11, 19:20:25, Epoch: 8, Loss: 1.0672e+05, c1: 3.1405e+02, c2: 4.1274e-01, c3: 5.5105e-01, dr1: 3.1414e+02, dr2: 4.1576e-01, dr3: 5.4738e-01, 					  dv: 1.0577e-03, ir1: 3.1672e+02, ir2: 4.1370e-01, ir3: 5.5536e-01, iv: 1.1225e-03
2020/07/11, 19:21:04, Epoch: 10, Loss: 8.3559e+0

2020/07/11, 19:33:26, Epoch: 48, Loss: 3.3192e+04, c1: 1.6525e+02, c2: 2.5187e-01, c3: 2.5267e-01, dr1: 1.6291e+02, dr2: 2.5093e-01, dr3: 2.5295e-01, 					  dv: 3.2695e-04, ir1: 1.6732e+02, ir2: 2.5246e-01, ir3: 2.5183e-01, iv: 1.3488e-04
2020/07/11, 19:34:05, Epoch: 50, Loss: 3.2244e+04, c1: 1.5582e+02, c2: 2.4876e-01, c3: 2.4401e-01, dr1: 1.5370e+02, dr2: 2.4815e-01, dr3: 2.4521e-01, 					  dv: 3.1775e-04, ir1: 1.5834e+02, ir2: 2.5009e-01, ir3: 2.4318e-01, iv: 1.2592e-04
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/0707_two_values_of_c_second_order_equation/1_adam/checkpoint_50/direct_model/assets
INFO:tensorflow:Assets written to: models/0707_two_values_of_c_second_order_equation/1_adam/checkpoint_50/inverse_model/assets
2020/07/11, 19:34:44, Epoch: 52, Loss: 3.2005e+04, c1: 1.4654e+02, c2: 2.4083e-01, c3: 2.3564e-01, dr1: 1.4499e+02, dr2: 2.4018e-01, dr3: 2.3614e-01, 					  dv: 3.1564e-04, ir1: 1.4858e+02

2020/07/11, 19:53:38, Epoch: 110, Loss: 2.5818e+04, c1: 6.5511e+00, c2: 1.8344e-02, c3: 1.8894e-02, dr1: 6.5479e+00, dr2: 1.8349e-02, dr3: 1.8901e-02, 					  dv: 2.5799e-04, ir1: 6.5506e+00, ir2: 1.8329e-02, ir3: 1.8798e-02, iv: 5.8724e-05
2020/07/11, 19:54:17, Epoch: 112, Loss: 2.5903e+04, c1: 6.5616e+00, c2: 1.7980e-02, c3: 1.9431e-02, dr1: 6.5851e+00, dr2: 1.8044e-02, dr3: 1.9357e-02, 					  dv: 2.5883e-04, ir1: 6.5703e+00, ir2: 1.7987e-02, ir3: 1.9373e-02, iv: 5.8844e-05
2020/07/11, 19:54:56, Epoch: 114, Loss: 2.5822e+04, c1: 6.6178e+00, c2: 1.8669e-02, c3: 1.9242e-02, dr1: 6.6215e+00, dr2: 1.8658e-02, dr3: 1.9198e-02, 					  dv: 2.5802e-04, ir1: 6.6473e+00, ir2: 1.8608e-02, ir3: 1.9173e-02, iv: 5.8120e-05
2020/07/11, 19:55:35, Epoch: 116, Loss: 2.5771e+04, c1: 6.6647e+00, c2: 1.8958e-02, c3: 1.9227e-02, dr1: 6.6540e+00, dr2: 1.8961e-02, dr3: 1.9088e-02, 					  dv: 2.5751e-04, ir1: 6.6547e+00, ir2: 1.8966e-02, ir3: 1.9167e-02, iv: 5.8011e-05
2020/07/11, 19:56:14, Epoch: 118, Loss: 

2020/07/11, 20:15:43, Epoch: 178, Loss: 2.5085e+04, c1: 7.6946e+00, c2: 2.3777e-02, c3: 2.1402e-02, dr1: 7.6199e+00, dr2: 2.3758e-02, dr3: 2.1239e-02, 					  dv: 2.5062e-04, ir1: 7.6525e+00, ir2: 2.3792e-02, ir3: 2.1413e-02, iv: 6.0725e-05
2020/07/11, 20:16:22, Epoch: 180, Loss: 2.5305e+04, c1: 7.5253e+00, c2: 2.3244e-02, c3: 2.0915e-02, dr1: 7.4830e+00, dr2: 2.3284e-02, dr3: 2.0860e-02, 					  dv: 2.5283e-04, ir1: 7.4725e+00, ir2: 2.3365e-02, ir3: 2.1022e-02, iv: 6.1034e-05
2020/07/11, 20:17:01, Epoch: 182, Loss: 2.5319e+04, c1: 7.5660e+00, c2: 2.3642e-02, c3: 2.1166e-02, dr1: 7.4785e+00, dr2: 2.3511e-02, dr3: 2.1083e-02, 					  dv: 2.5296e-04, ir1: 7.5117e+00, ir2: 2.3663e-02, ir3: 2.1009e-02, iv: 6.1314e-05
2020/07/11, 20:17:41, Epoch: 184, Loss: 2.5153e+04, c1: 7.5783e+00, c2: 2.3766e-02, c3: 2.0777e-02, dr1: 7.5367e+00, dr2: 2.3724e-02, dr3: 2.0626e-02, 					  dv: 2.5130e-04, ir1: 7.5690e+00, ir2: 2.3685e-02, ir3: 2.0802e-02, iv: 6.1089e-05
2020/07/11, 20:18:20, Epoch: 186, Loss: 

2020/07/11, 20:37:51, Epoch: 246, Loss: 2.4376e+04, c1: 9.2872e+00, c2: 2.4246e-02, c3: 2.6815e-02, dr1: 9.1360e+00, dr2: 2.4423e-02, dr3: 2.6993e-02, 					  dv: 2.4348e-04, ir1: 9.3753e+00, ir2: 2.4277e-02, ir3: 2.6660e-02, iv: 6.3880e-05
2020/07/11, 20:38:30, Epoch: 248, Loss: 2.4268e+04, c1: 9.1925e+00, c2: 2.6090e-02, c3: 2.5436e-02, dr1: 9.0455e+00, dr2: 2.6093e-02, dr3: 2.5383e-02, 					  dv: 2.4240e-04, ir1: 9.3027e+00, ir2: 2.6151e-02, ir3: 2.5317e-02, iv: 6.2982e-05
2020/07/11, 20:39:10, Epoch: 250, Loss: 2.4304e+04, c1: 9.0651e+00, c2: 2.5647e-02, c3: 2.5889e-02, dr1: 8.9232e+00, dr2: 2.5700e-02, dr3: 2.5927e-02, 					  dv: 2.4277e-04, ir1: 9.1992e+00, ir2: 2.5745e-02, ir3: 2.5748e-02, iv: 6.4137e-05
INFO:tensorflow:Assets written to: models/0707_two_values_of_c_second_order_equation/1_adam/checkpoint_250/direct_model/assets
INFO:tensorflow:Assets written to: models/0707_two_values_of_c_second_order_equation/1_adam/checkpoint_250/inverse_model/assets
2020/07/11, 20:39:49, Epo

2020/07/11, 20:59:21, Epoch: 312, Loss: 2.3267e+04, c1: 5.9603e+00, c2: 1.7230e-02, c3: 2.0816e-02, dr1: 5.8861e+00, dr2: 1.7052e-02, dr3: 2.0758e-02, 					  dv: 2.3249e-04, ir1: 5.9934e+00, ir2: 1.7218e-02, ir3: 2.0714e-02, iv: 6.7960e-05
2020/07/11, 21:00:00, Epoch: 314, Loss: 2.3288e+04, c1: 6.3025e+00, c2: 1.9339e-02, c3: 2.2131e-02, dr1: 6.2500e+00, dr2: 1.9287e-02, dr3: 2.2179e-02, 					  dv: 2.3269e-04, ir1: 6.3637e+00, ir2: 1.9369e-02, ir3: 2.2022e-02, iv: 6.8572e-05
2020/07/11, 21:00:39, Epoch: 316, Loss: 2.3065e+04, c1: 6.3191e+00, c2: 1.9130e-02, c3: 2.2582e-02, dr1: 6.2852e+00, dr2: 1.9287e-02, dr3: 2.2629e-02, 					  dv: 2.3046e-04, ir1: 6.4252e+00, ir2: 1.9039e-02, ir3: 2.2466e-02, iv: 6.7527e-05
2020/07/11, 21:01:18, Epoch: 318, Loss: 2.3038e+04, c1: 6.2718e+00, c2: 1.9186e-02, c3: 2.2060e-02, dr1: 6.2404e+00, dr2: 1.9120e-02, dr3: 2.1903e-02, 					  dv: 2.3019e-04, ir1: 6.3661e+00, ir2: 1.9104e-02, ir3: 2.2161e-02, iv: 6.7713e-05
2020/07/11, 21:01:57, Epoch: 320, Loss: 

2020/07/11, 21:21:26, Epoch: 380, Loss: 2.1878e+04, c1: 1.0048e+01, c2: 3.1748e-02, c3: 3.1820e-02, dr1: 1.0074e+01, dr2: 3.2103e-02, dr3: 3.1971e-02, 					  dv: 2.1848e-04, ir1: 9.9519e+00, ir2: 3.2082e-02, ir3: 3.2281e-02, iv: 7.5546e-05
2020/07/11, 21:22:04, Epoch: 382, Loss: 2.1676e+04, c1: 9.9290e+00, c2: 3.1924e-02, c3: 3.0883e-02, dr1: 9.9632e+00, dr2: 3.2086e-02, dr3: 3.1133e-02, 					  dv: 2.1646e-04, ir1: 9.9227e+00, ir2: 3.2120e-02, ir3: 3.1324e-02, iv: 7.4411e-05
2020/07/11, 21:22:43, Epoch: 384, Loss: 2.1522e+04, c1: 9.8312e+00, c2: 3.1616e-02, c3: 3.0467e-02, dr1: 9.8554e+00, dr2: 3.1790e-02, dr3: 3.0620e-02, 					  dv: 2.1493e-04, ir1: 9.8139e+00, ir2: 3.1756e-02, ir3: 3.0750e-02, iv: 7.3587e-05
2020/07/11, 21:23:22, Epoch: 386, Loss: 2.1576e+04, c1: 9.7318e+00, c2: 3.0515e-02, c3: 3.1104e-02, dr1: 9.7478e+00, dr2: 3.0900e-02, dr3: 3.1285e-02, 					  dv: 2.1546e-04, ir1: 9.6897e+00, ir2: 3.0611e-02, ir3: 3.1401e-02, iv: 7.4559e-05
2020/07/11, 21:24:01, Epoch: 388, Loss: 

In [ ]:
i = 2
optimizer = keras.optimizers.Adam(learning_rate = keras.optimizers.schedules.PiecewiseConstantDecay([1000], 
										[1e-4, 1e-5]))

w_c1, w_c2, w_c3 = 1.0, 1.0, 1.0
w_dr1, w_dr2, w_dr3 = 1.0, 1.0, 1.0
w_ir1, w_ir2, w_ir3 = 1.0, 1.0, 1.0
w_dv = 1e5
w_iv = 1e2

w_zero = 0.1
threshold = 1e-3

train(True, data_c, data_i, data_d, 
      start_epoch = 0, epochs = 2000, 
      batch_proportion = 0.1, print_every = 2, save_every = 50, save_path = "{}_adam".format(i))

In [ ]:
x_test = np.linspace(1.0, 500.0, 500) / x0
z_test = np.linspace(1.0, 500.0, 500) / z0
xz_test = tensor_grid([z_test, x_test])
xz_test[:, [0, 1]] = xz_test[:, [1, 0]]

c_pred = inverse_model(tf.constant(xz_test, dtype = tf.float32), training = False)*c0
def c_func(x, z):
	return np.piecewise(z, [z > 300, (z >= 200) & (z <= 300), z < 200], [1500, 3500, 1500])
c_true = c_func(xz_test[:, 0:1]*x0, xz_test[:, 1:2]*z0)

import matplotlib as mpl
from matplotlib.cm import cool
from matplotlib.colors import Normalize

from mpl_toolkits.axes_grid1 import make_axes_locatable

cmap = cool
norm = Normalize(vmin=1000, vmax=5000)

X, Z = np.meshgrid(x_test, z_test)
fig, ax = plt.subplots(1, 3, figsize = (15, 5))
fig.subplots_adjust(right = 1.0)

im0 = ax[0].contourf(X, Z, c_true.reshape((len(z_test), len(x_test))), cmap = cmap, norm = norm)
ax[0].set_title("true")
divider = make_axes_locatable(ax[0])
cax = divider.append_axes('right', size='5%', pad=0.05)
mpl.colorbar.ColorbarBase(cax, cmap = cmap, norm = norm, orientation='vertical')

norm = mpl.colors.Normalize(vmin=0, vmax=5000)
im1 = ax[1].contourf(X, Z, c_pred.numpy().reshape((len(z_test), len(x_test))), cmap = cmap, norm = norm)
ax[1].set_title("pred")
ax[1].set_xlabel("")
divider = make_axes_locatable(ax[1])
cax = divider.append_axes('right', size='5%', pad=0.05)
mpl.colorbar.ColorbarBase(cax, cmap = cmap, norm = norm, orientation='vertical')

norm = mpl.colors.Normalize(vmin=-1e-2, vmax=1e-2)
im2 = ax[2].contourf(X, Z, c_true.reshape((len(z_test), len(x_test)))-c_pred.numpy().reshape((len(z_test), len(x_test))), cmap = cmap, norm = norm)
ax[2].set_title("error")
divider = make_axes_locatable(ax[2])
cax = divider.append_axes('right', size='5%', pad=0.05)
mpl.colorbar.ColorbarBase(cax, cmap = cmap, norm = norm, orientation='vertical')

plt.show()

In [ ]:
np.max(c_pred)

In [ ]:
np.min(c_pred)

In [ ]:
i = 5
p = loadmat("data/sr_{}.mat".format(int(x_s[i]*x0/dx)))["sr_{}".format(int(x_s[i]*x0/dx))]

x_test = np.linspace(10.0, 500.0, 50) / x0
z_test = np.array([10]) / z0
t_test = np.linspace(0.0, 0.49, 50) / t0
xzt_test = tensor_grid([t_test, x_test, z_test])
xzt_test[:, [0, 1, 2]] = xzt_test[:, [1, 2, 0]]

xzts = np.hstack([xzt_test, x_s[i]*np.ones_like(xzt_test[:, 0:1])])
out = direct_model(tf.constant(xzts, dtype = tf.float32))

cmap = cool
norm = Normalize(vmin=-0.2, vmax=0.2)

X, T = np.meshgrid(x_test, t_test)
fig, ax = plt.subplots(1, 3, figsize = (15, 5))
fig.subplots_adjust(right = 1.0)

im0 = ax[0].contourf(X, T, p, cmap = cmap, norm = norm, levels = 1000)
ax[0].set_title("p, true")
divider = make_axes_locatable(ax[0])
cax = divider.append_axes('right', size='5%', pad=0.05)
mpl.colorbar.ColorbarBase(cax, cmap = cmap, norm = norm, orientation='vertical')

norm = Normalize(vmin=-0.2, vmax=0.2)
im1 = ax[1].contourf(X, T, out[:, 0:1].numpy().reshape((len(t_test), len(x_test))), cmap = cmap, norm = norm, levels = 1000)
ax[1].set_title("p, pred")
ax[1].set_xlabel("")
divider = make_axes_locatable(ax[1])
cax = divider.append_axes('right', size='5%', pad=0.05)
mpl.colorbar.ColorbarBase(cax, cmap = cmap, norm = norm, orientation='vertical')

norm = mpl.colors.Normalize(vmin=-1e-3, vmax=1e-3)
im2 = ax[2].contourf(X, T, p-out[:, 0:1].numpy().reshape((len(t_test), len(x_test))), cmap = cmap, norm = norm, levels = 1000)
ax[2].set_title("p, error")
divider = make_axes_locatable(ax[2])
cax = divider.append_axes('right', size='5%', pad=0.05)
mpl.colorbar.ColorbarBase(cax, cmap = cmap, norm = norm, orientation='vertical')

plt.show()

In [ ]:
i = 0

x_test = np.linspace(10.0, 500.0, 50) / x0
z_test = np.linspace(10.0, 500.0, 50) / z0
t_test = np.array([0]) / t0
xzt_test = tensor_grid([t_test, x_test, z_test])
xzt_test[:, [0, 1, 2]] = xzt_test[:, [1, 2, 0]]

xzts = np.hstack([xzt_test, x_s[i]*np.ones_like(xzt_test[:, 0:1])])
out = direct_model(tf.constant(xzts, dtype = tf.float32))

cmap = cool
norm = Normalize(vmin=-0.1, vmax=0.1)

X, Z = np.meshgrid(x_test, z_test)
fig, ax = plt.subplots(1, 1, figsize = (5, 5))
fig.subplots_adjust(right = 1.0)

im0 = ax.contourf(X, Z, out[:, 0:1].numpy().reshape((len(z_test), len(x_test))), cmap = cmap, norm = norm, levels = 1000)
ax.set_title("p, pred, t=0")
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size='5%', pad=0.05)
mpl.colorbar.ColorbarBase(cax, cmap = cmap, norm = norm, orientation='vertical')

plt.show()